In [15]:
import pandas as pd
import numpy as np
from sklearn.impute import KNNImputer
from statsmodels.tsa.arima.model import ARIMA 
import matplotlib.pyplot as plt

# Data visualization

In [17]:
# Load the data
file_path = "../data/raw/final_merged_data.csv"
data = pd.read_csv(file_path)

In [ ]:
# Replace -1 with NaN for processing
columns_with_nan = ["inside_temperature", "inside_humidity"]
data[columns_with_nan] = data[columns_with_nan].replace(-1, np.nan)
print(data[columns_with_nan].isna().sum())

In [ ]:
print(data[columns_with_nan].head(10))
mask = data[columns_with_nan].isna()
print(mask.sum())


In [ ]:
# Analyze and Visualize the Data
print("Basic Dataset Information:")
print(data.info())
print("\nSummary Statistics:")
print(data.describe())

In [ ]:
# Count and visualize missing values
missing_counts = data.isna().sum()
print("\nMissing Values per Column:")
print(missing_counts)

In [ ]:
plt.figure(figsize=(8, 4))
missing_counts.plot(kind='bar', color='skyblue')
plt.title("Missing Values per Column")
plt.ylabel("Count")
plt.show()

In [ ]:
# Visualize the data distribution
for col in columns_with_nan:
    plt.figure(figsize=(8, 4))
    data[col].plot(kind='hist', bins=50, color='lightgreen', edgecolor='black')
    plt.title(f"Distribution of {col}")
    plt.xlabel(col)
    plt.ylabel("Frequency")
    plt.show()

In [ ]:
from tabulate import tabulate

def analyze_missing_gaps(data, columns_with_nan):
    gaps_info = []
    for col in columns_with_nan:
        data['gap'] = data[col].isna()
        data['gap_id'] = (data['gap'] != data['gap'].shift()).cumsum()
        
        missing_gaps = data[data['gap']].groupby('gap_id')
        
        for gap_id, gap_data in missing_gaps:
            start_time = gap_data['time'].iloc[0]
            end_time = gap_data['time'].iloc[-1]
            gap_count = len(gap_data)
            
            # Calculate duration of the gap
            gap_duration = (pd.to_datetime(end_time) - pd.to_datetime(start_time)) + pd.Timedelta(minutes=1)
            
            # Format duration as '0 days HH:mm:ss'
            gap_duration_str = str(gap_duration)
            
            gaps_info.append({
                "Column": col,
                "Start Time": start_time,
                "End Time": end_time,
                "Duration": gap_duration_str,
            })
    
    gaps_df = pd.DataFrame(gaps_info)
    
    # Count the frequency of each unique duration for each column
    frequency_summary = gaps_df.groupby(['Column', 'Duration']).size().reset_index(name='Count')
    return frequency_summary


# Generate gap analysis and display results
gaps_summary = analyze_missing_gaps(data, columns_with_nan)

print("Missing Gaps Summary:")
print(tabulate(gaps_summary, headers='keys', tablefmt='pretty'))


for col in columns_with_nan:
    col_summary = gaps_summary[gaps_summary["Column"] == col]
    print(f"Aggregated Missing Gaps Summary for {col}:")
    print(tabulate(col_summary, headers="keys", tablefmt="pretty"))
    print("\n")

# Plot gaps frequency for each column
for col in columns_with_nan:
    col_gaps = gaps_summary[gaps_summary['Column'] == col]
    
    col_gaps['Duration Hours'] = pd.to_timedelta(col_gaps['Duration']).dt.total_seconds() / 3600

    duration_counts = col_gaps['Duration Hours'].value_counts().reset_index()
    duration_counts.columns = ['Duration (hours)', 'Frequency']
    duration_counts.sort_values(by='Duration (hours)', inplace=True)

    plt.figure(figsize=(10, 6))
    plt.bar(col_gaps['Duration'], col_gaps['Count'], color='skyblue', edgecolor='black')
    plt.xlabel("Gap Duration (hours)")
    plt.ylabel("Frequency")
    plt.title("Frequency of Gap Durations")
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.show()



# Imputation methods

In [65]:
def evaluate_imputation(original, imputed, method):
    mask = ~original.isna()
    mse = np.mean((original[mask] - imputed[mask])**2)
    print(f"{method} MSE: {mse}")
    return mse


In [ ]:
original_data = data.dropna(subset=columns_with_nan).copy()
mask = data[columns_with_nan].isna()
print(original_data.isna().sum())


## Statistical Methods

In [ ]:
# Mean Imputation
mean_imputed = data.copy()
for col in columns_with_nan:
    mean_imputed[col].fillna(mean_imputed[col].mean(), inplace=True)

In [ ]:
# Median Imputation
median_imputed = data.copy()
for col in columns_with_nan:
    median_imputed[col].fillna(median_imputed[col].median(), inplace=True)

In [ ]:
# LOCF (Last Observation Carried Forward)
locf_imputed = data.copy()
locf_imputed[columns_with_nan] = locf_imputed[columns_with_nan].fillna(method='ffill')

In [ ]:
# NOCB (Next Observation Carried Backward)
nocb_imputed = data.copy()
nocb_imputed[columns_with_nan] = nocb_imputed[columns_with_nan].fillna(method='bfill')

## Interpolation Methods

In [123]:
# Linear Interpolation
linear_imputed = data.copy()
linear_imputed[columns_with_nan] = linear_imputed[columns_with_nan].interpolate(method='linear')

In [ ]:
# Spline Interpolation
spline_imputed = data.copy()
spline_imputed[columns_with_nan] = spline_imputed[columns_with_nan].interpolate(method='spline', order=3)


## Machine Learning Methods

In [39]:
# KNN Imputation
knn_imputer = KNNImputer(n_neighbors=60) # 60 neighbors to look up at the 1 hour interval
knn_imputed = data.copy()
knn_imputed[columns_with_nan] = knn_imputer.fit_transform(knn_imputed[columns_with_nan])

# ARIMA-based methods

In [27]:
def arima_imputation(series):
    series_imputed = series.copy()
    for i in range(len(series)):
        if pd.isna(series[i]):
            model = ARIMA(series[:i].dropna(), order=(1, 1, 1))
            model_fit = model.fit()
            series_imputed[i] = model_fit.forecast()[0]
    return series_imputed

In [ ]:
arima_imputed = data.copy()
arima_imputed["inside_temperature"] = arima_imputation(data["inside_temperature"])
arima_imputed["inside_humidity"] = arima_imputation(data["inside_humidity"])

# Compare Techniques

In [ ]:
print(data[columns_with_nan].isna().sum())
print(linear_imputed[columns_with_nan].isna().sum())
print(mean_imputed[columns_with_nan].isna().sum())
print(median_imputed[columns_with_nan].isna().sum())
print(locf_imputed[columns_with_nan].isna().sum())
print(nocb_imputed[columns_with_nan].isna().sum())
print(knn_imputed[columns_with_nan].isna().sum())


In [ ]:
print('Knn')
print(knn_imputed.describe())
print('linear_imputed')

print(linear_imputed.describe())
print('mean_imputed')

print(mean_imputed.describe())
print('median_imputed')

print(median_imputed.describe())
print('locf_imputed')

print(locf_imputed.describe())
print('nocb_imputed')

print(nocb_imputed.describe())
print('data')


print(data.describe())

In [ ]:
for col in columns_with_nan:
    plt.figure(figsize=(8, 4))
    nocb_imputed[col].plot(kind='hist', bins=50, color='lightgreen', edgecolor='black')
    plt.title(f"Distribution of {col}")
    plt.xlabel(col)
    plt.ylabel("Frequency")
    plt.show()

In [ ]:
for col in columns_with_nan:
    plt.figure(figsize=(8, 4))
    locf_imputed[col].plot(kind='hist', bins=50, color='lightgreen', edgecolor='black')
    plt.title(f"Distribution of {col}")
    plt.xlabel(col)
    plt.ylabel("Frequency")
    plt.show()

In [ ]:
for col in columns_with_nan:
    plt.figure(figsize=(8, 4))
    median_imputed[col].plot(kind='hist', bins=50, color='lightgreen', edgecolor='black')
    plt.title(f"Distribution of {col}")
    plt.xlabel(col)
    plt.ylabel("Frequency")
    plt.show()

In [ ]:
for col in columns_with_nan:
    plt.figure(figsize=(8, 4))
    mean_imputed[col].plot(kind='hist', bins=50, color='lightgreen', edgecolor='black')
    plt.title(f"Distribution of {col}")
    plt.xlabel(col)
    plt.ylabel("Frequency")
    plt.show()

In [ ]:
for col in columns_with_nan:
    plt.figure(figsize=(8, 4))
    linear_imputed[col].plot(kind='hist', bins=50, color='lightgreen', edgecolor='black')
    plt.title(f"Distribution of {col}")
    plt.xlabel(col)
    plt.ylabel("Frequency")
    plt.show()

In [ ]:
### Custom imputation/interpolation algorithm

In [35]:
import pandas as pd
from statsmodels.tsa.seasonal import seasonal_decompose

def impute_missing_gaps_independently(data, zone_column, time_column, value_column):
    """
    Impute missing gaps independently for each zone and time.
    """
    data = data.copy()
    data[time_column] = pd.to_datetime(data[time_column])
    data = data.sort_values(by=[zone_column, time_column])

    processed_zones = []

    for zone, zone_data in data.groupby(zone_column):
        zone_data = zone_data.copy()
        zone_data['gap'] = zone_data[value_column].isna()
        zone_data['gap_id'] = (zone_data['gap'] != zone_data['gap'].shift()).cumsum()

        for gap_id, gap_data in zone_data.groupby('gap_id'):
            if not gap_data['gap'].iloc[0]:
                continue

            gap_mask = (zone_data['gap_id'] == gap_id) & zone_data['gap']
            start_time = gap_data[time_column].iloc[0]
            end_time = gap_data[time_column].iloc[-1]
            gap_duration = (end_time - start_time).total_seconds() / 60

            if gap_duration <= 10:
                temp_series = pd.Series(zone_data[value_column].values,
                                      index=zone_data[time_column])
                filled_values = temp_series.interpolate(method='linear')
                zone_data.loc[gap_mask, value_column] = filled_values[gap_data[time_column]].values
            elif 10 < gap_duration <= 60:
                temp_series = pd.Series(zone_data[value_column].values,
                                      index=zone_data[time_column])
                filled_values = temp_series.interpolate(method='time')
                zone_data.loc[gap_mask, value_column] = filled_values[gap_data[time_column]].values
            elif gap_duration > 60:
                # Create a time series with only non-null values
                valid_data = zone_data[~zone_data[value_column].isna()]
                if len(valid_data) < 1440:  # If not enough data for seasonal decomposition
                    temp_series = pd.Series(zone_data[value_column].values,
                                          index=zone_data[time_column])
                    filled_values = temp_series.interpolate(method='time')
                    zone_data.loc[gap_mask, value_column] = filled_values[gap_data[time_column]].values
                else:
                    temp_series = pd.Series(valid_data[value_column].values,
                                          index=valid_data[time_column])
                    try:
                        decomposed = seasonal_decompose(temp_series, period=1440)
                        trend = pd.Series(decomposed.trend, index=temp_series.index)
                        trend = trend.ffill().bfill()
                        # Reindex trend to match gap dates
                        trend_reindexed = trend.reindex(gap_data[time_column])
                        zone_data.loc[gap_mask, value_column] = trend_reindexed.values
                    except:
                        # Fallback to time-based interpolation if decomposition fails
                        temp_series = pd.Series(zone_data[value_column].values,
                                              index=zone_data[time_column])
                        filled_values = temp_series.interpolate(method='time')
                        zone_data.loc[gap_mask, value_column] = filled_values[gap_data[time_column]].values
            else:
                zone_data.loc[gap_mask, value_column] = zone_data[value_column].ffill()

        zone_data = zone_data.drop(columns=['gap', 'gap_id'])
        processed_zones.append(zone_data)

    result = pd.concat(processed_zones)
    return result

cleaned_data = impute_missing_gaps_independently(data, 'zone', 'time', 'inside_temperature')
cleaned_data = impute_missing_gaps_independently(cleaned_data, 'zone', 'time', 'inside_humidity')
cleaned_data = impute_missing_gaps_independently(cleaned_data, 'zone', 'time', 'GHI')
cleaned_data = impute_missing_gaps_independently(cleaned_data, 'zone', 'time', 'DNI')
cleaned_data = impute_missing_gaps_independently(cleaned_data, 'zone', 'time', 'DHI')
cleaned_data = impute_missing_gaps_independently(cleaned_data, 'zone', 'time', 'outside_temperature')
cleaned_data = impute_missing_gaps_independently(cleaned_data, 'zone', 'time', 'outside_humidity')
cleaned_data = impute_missing_gaps_independently(cleaned_data, 'zone', 'time', 'BP')
cleaned_data = impute_missing_gaps_independently(cleaned_data, 'zone', 'time', 'WS')
cleaned_data = impute_missing_gaps_independently(cleaned_data, 'zone', 'time', 'WD_Avg')
cleaned_data = impute_missing_gaps_independently(cleaned_data, 'zone', 'time', 'WSgust_Max')

In [ ]:
print(cleaned_data.info())

In [ ]:
cleaned_data.describe()

In [41]:
cleaned_data.to_csv('../data/processed/imputedData.csv', index=False)

In [ ]:
cleaned_data['inside_temperature', 'inside_humidity'].describe()

In [ ]:
for col in columns_with_nan:
    plt.figure(figsize=(8, 4))
    cleaned_data[col].plot(kind='hist', bins=50, color='lightgreen', edgecolor='black')
    plt.title(f"Distribution of {col}")
    plt.xlabel(col)
    plt.ylabel("Frequency")
    plt.show()

In [ ]:
for col in columns_with_nan:
    plt.figure(figsize=(8, 4))
    data[col].plot(kind='hist', bins=50, color='lightgreen', edgecolor='black')
    plt.title(f"Distribution of {col}")
    plt.xlabel(col)
    plt.ylabel("Frequency")
    plt.show()

In [ ]:
columns = ['inside_temperature', 'inside_humidity','GHI','DNI','DHI','outside_temperature','outside humidity','BP','WS','WD_Avg','WSgust_Max']  

for col in columns:
    plt.figure(figsize=(8, 6))
    plt.hist(data[col], bins=30, alpha=0.5, label='Original', color = 'blue')
    plt.hist(cleaned_data[col], bins=30, alpha=0.5, label='Cleaned', color = 'green')
    plt.title(f'Distribution of {col}')
    plt.legend()
    plt.show()
